In [1]:
from fpl_helpers import *
import pandas as pd

In [2]:
elements_df, teams_df = get_tables()
global PLAYER_IDS
PLAYER_IDS = elements_df[['full_name', 'id']].set_index('full_name').to_dict()['id']
PLAYER_NAMES = {PLAYER_IDS[i]:i for i in list(PLAYER_IDS.keys()) }


In [3]:
def get_team(team_id, gw):
  url = f"https://fantasy.premierleague.com/api/entry/{team_id}/event/{gw}/picks/"
  r = requests.get(url)
  json = r.json()
  return json

team_ids = [i['element'] for i in get_team(715692, 26)['picks']]
team_names = [PLAYER_NAMES[i] for i in team_ids]

In [4]:
team_names

['Jordan Pickford',
 'Gabriel dos Santos Magalhães',
 'Daniel Muñoz',
 'Andrew Robertson',
 'Morgan Rogers',
 'Justin Kluivert',
 'Cole Palmer',
 'Mohamed Salah',
 'Chris Wood',
 'Ollie Watkins',
 'Jean-Philippe Mateta',
 'Hákon Valdimarsson',
 'Anthony Gordon',
 'Lewis Hall',
 'Taylor Harwood-Bellis']

In [5]:
compare_players(team_names[5:11], metrics=ATTACKER_METRICS, agg='mean')

,Difficulty,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected
Justin Kluivert,"3.0, 3.0, 2.0, 2.0, 2.0, 3.0",3.00,0.00,0.34,0.33,0.06,0.39,0.00,10.67,16.07,30.23,25.67,7.20,1.00,2655337.67
Cole Palmer,"2.0, 2.0, 4.0, 3.0, 3.0, 2.0",3.33,0.00,0.29,0.33,0.29,0.58,0.67,17.67,9.53,26.70,19.00,5.53,1.00,7103578.67
Mohamed Salah,"3.0, 2.0, 2.0, 3.0, 2.0, 2.0",11.33,1.00,0.50,0.67,0.16,0.66,2.00,38.33,51.07,25.17,36.00,11.23,1.00,8130280.67
Chris Wood,"4.0, 4.0, 2.0, 3.0, 3.0, 2.0",10.33,1.33,0.77,0.33,0.24,1.01,2.00,46.33,54.67,19.00,41.33,11.53,1.00,4876796.33
Ollie Watkins,"3.0, 3.0, 3.0, 3.0, 2.0, 3.0",5.67,0.67,0.60,0.00,0.04,0.65,1.00,20.67,24.40,4.43,41.67,7.03,1.00,2336077.00
Jean-Philippe Mateta,"3.0, 2.0, 2.0, 3.0, 4.0, 4.0",8.33,1.00,1.01,0.33,0.29,1.31,1.33,34.67,49.53,20.00,37.67,10.73,1.00,830645.67


In [6]:
forwards = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==4)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
forwards['difficulty'] = forwards['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
forwards['avg_difficulty'] = forwards['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
forwards['cost_per_point'] = forwards['now_cost']/forwards['total_points']
#forwards[:20].sort_values(by='difficulty')
forwards

,full_name,team,selected_by_percent,now_cost,value_season,total_points,expected_goals,goals_scored,expected_assists,assists,expected_goals_conceded,goals_conceded,difficulty,avg_difficulty,cost_per_point
606,Chris Wood,Nott'm Forest,44.6,72,23.3,168,10.33,18,1.06,3,27.98,29,"[4.0, 4.0, 2.0, 3.0, 3.0, 2.0]",3.000000,0.428571
557,Alexander Isak,Newcastle,57.8,95,17.6,167,15.09,19,3.12,5,28.37,33,"[5.0, 2.0, 2.0, 2.0, 3.0, 3.0]",2.833333,0.568862
478,Erling Haaland,Man City,26.3,147,10.5,154,18.30,19,1.66,3,35.00,35,"[3.0, 4.0, 3.0, 2.0, 3.0, 3.0]",3.000000,0.954545
747,Matheus Santos Carneiro Da Cunha,Wolves,14.5,69,20.4,141,6.47,13,4.36,4,35.93,47,"[3.0, 2.0, 2.0, 2.0, 2.0, 3.0]",2.333333,0.489362
64,Ollie Watkins,Aston Villa,20.4,89,14.9,133,13.05,12,1.52,5,27.39,33,"[3.0, 3.0, 3.0, 3.0, 2.0, 3.0]",2.833333,0.669173
269,Jean-Philippe Mateta,Crystal Palace,9.6,74,15.5,115,11.03,11,2.10,2,29.98,27,"[3.0, 2.0, 2.0, 3.0, 4.0, 4.0]",3.000000,0.643478
143,Yoane Wissa,Brentford,13.6,64,17.7,113,12.04,11,1.27,3,30.44,30,"[2.0, 3.0, 3.0, 4.0, 4.0, 4.0]",3.333333,0.566372
233,Nicolas Jackson,Chelsea,7.0,77,14.0,108,10.71,9,1.65,6,23.95,19,"[2.0, 2.0, 4.0, 3.0, 3.0, 2.0]",2.666667,0.712963
341,Raúl Jiménez,Fulham,8.3,56,19.1,107,9.19,9,1.72,3,20.91,24,"[2.0, 3.0, 3.0, 4.0, 5.0, 3.0]",3.333333,0.523364
366,Liam Delap,Ipswich,3.6,56,18.4,103,7.91,10,1.28,2,40.15,41,"[3.0, 3.0, 3.0, 3.0, 2.0, 4.0]",3.000000,0.543689


In [7]:
forward_scores = compare_players(forwards['full_name'], metrics=ATTACKER_METRICS, agg='mean')
forward_scores.data['Average Difficulty'] = forward_scores.data['Difficulty'].apply(lambda x: np.mean(x))
#forward_scores

In [8]:
preferred_low = ["Average Difficulty"]

In [9]:
weights = {
    'Average Difficulty':-2,
    'Total Points':1.5, 
    'Goals Scored': 2,
    'Expected Goals':3,
    'Assists':0,
    'Expected Assists':0, 
    'Expected Goal Involvements':1, 
    'Bonus':1.5,
    'Bps':1.5,
    'Influence':0, 
    'Creativity':0, 
    'Threat': 0,
    'Ict Index':0, 
    'Starts':10
}
df = calculate_weighted_score(forward_scores.data.drop('Difficulty', axis=1), weights, preferred_low)
df

,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected,Average Difficulty,total_score
Chris Wood,10.33,1.33,0.77,0.33,0.24,1.01,2.00,46.33,54.67,19.00,41.33,11.53,1.00,4876796.33,3.00,17.85
Matheus Santos Carneiro Da Cunha,8.33,1.00,0.54,0.00,0.22,0.76,2.67,36.33,43.87,30.70,37.00,11.13,1.00,970199.67,2.33,17.58
Jean-Philippe Mateta,8.33,1.00,1.01,0.33,0.29,1.31,1.33,34.67,49.53,20.00,37.67,10.73,1.00,830645.67,3.00,17.26
Norberto Bercique Gomes Betuncal,7.33,1.00,0.53,0.00,0.05,0.58,1.33,33.00,43.87,19.93,49.67,11.37,1.00,237460.67,3.33,14.40
Ollie Watkins,5.67,0.67,0.60,0.00,0.04,0.65,1.00,20.67,24.40,4.43,41.67,7.03,1.00,2336077.00,2.83,14.35
Liam Delap,6.33,0.67,0.48,0.00,0.09,0.58,1.67,22.67,24.13,11.47,38.00,7.33,1.00,401439.67,3.00,14.15
Alexander Isak,5.67,0.67,0.35,0.00,0.07,0.42,1.00,22.33,27.53,9.40,23.33,6.03,1.00,6652048.00,2.83,13.48
Yoane Wissa,4.00,0.33,0.58,0.00,0.03,0.61,0.67,14.67,13.00,4.30,30.33,4.77,1.00,1351580.00,3.33,12.13
Raúl Jiménez,5.00,0.33,0.34,0.33,0.06,0.41,0.67,17.33,22.33,17.80,47.33,8.73,1.00,949795.67,3.33,11.51
Jamie Vardy,1.67,0.00,0.12,0.00,0.01,0.13,0.00,9.67,6.13,6.07,16.00,2.83,1.00,618512.33,3.17,9.38


In [10]:
midfielders = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==3)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
midfielders['difficulty'] = midfielders['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
midfielders['avg_difficulty'] = midfielders['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
midfielders['cost_per_point'] = midfielders['now_cost']/midfielders['total_points']
#forwards[:20].sort_values(by='difficulty')
#midfielders

In [11]:
mid_weights = {
    'Average Difficulty': -3,
    'Total Points':1.5, 
    'Goals Scored': 2,
    'Expected Goals':2,
    'Assists':1,
    'Expected Assists':1, 
    'Expected Goal Involvements':1, 
    'Bonus':1.5,
    'Bps':1.5,
    'Influence':0, 
    'Creativity':0, 
    'Threat': 0,
    'Ict Index':0, 
    'Starts':1
}

midfielder_scores = compare_players(midfielders['full_name'], metrics=ATTACKER_METRICS, agg='mean')
midfielder_scores.data['Average Difficulty'] = midfielder_scores.data['Difficulty'].apply(lambda x: np.mean(x))
df = calculate_weighted_score(midfielder_scores.data.drop('Difficulty', axis=1), mid_weights, preferred_low)
df


,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected,Average Difficulty,total_score
Mohamed Salah,11.33,1.00,0.50,0.67,0.16,0.66,2.00,38.33,51.07,25.17,36.00,11.23,1.00,8130280.67,2.33,11.28
Jarrod Bowen,7.33,0.67,0.37,0.00,0.17,0.54,1.67,24.33,31.87,13.07,34.67,7.93,1.00,322299.33,3.00,6.12
Dominik Szoboszlai,5.67,0.33,0.17,0.33,0.23,0.40,0.67,19.33,23.93,17.10,24.33,6.53,1.00,317995.33,2.33,5.67
Jacob Murphy,5.67,0.67,0.48,0.00,0.05,0.53,0.33,19.33,26.93,5.00,31.33,6.30,1.00,493758.67,2.83,5.36
Morgan Gibbs-White,8.67,0.33,0.17,1.33,0.08,0.25,0.67,33.67,26.60,16.67,23.33,6.67,1.00,541894.67,3.00,4.82
Anthony Elanga,5.67,0.00,0.35,1.00,0.37,0.72,0.33,17.00,22.93,30.47,19.00,7.23,1.00,704082.67,3.00,4.63
Bryan Mbeumo,6.33,0.33,0.28,0.33,0.44,0.72,1.00,25.00,30.40,32.23,41.33,10.40,1.00,4043762.00,3.33,4.58
Mitoma Kaoru,6.33,0.67,0.38,0.00,0.02,0.40,0.67,19.33,28.80,3.67,20.67,5.33,1.00,718521.33,3.00,4.54
Cole Palmer,3.33,0.00,0.29,0.33,0.29,0.58,0.67,17.67,9.53,26.70,19.00,5.53,1.00,7103578.67,2.67,4.51
Dango Ouattara,3.67,0.33,0.27,0.00,0.13,0.40,0.00,13.67,23.60,22.37,25.33,7.13,1.00,359529.67,2.50,4.11


In [12]:
defenders = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==2)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
defenders['difficulty'] = defenders['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
defenders['avg_difficulty'] = defenders['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
defenders['cost_per_point'] = defenders['now_cost']/defenders['total_points']
defender_scores = compare_players(defenders['full_name'], metrics=DEFENDER_METRICS, agg='mean')
defender_scores.data['Average Difficulty'] = defender_scores.data['Difficulty'].apply(lambda x: np.mean(x))


In [13]:
def_weights ={
 'Average Difficulty': -2,
 'Total Points': 3,
 'Clean Sheets': 1,
 'Goals Conceded': -1,
 'Own Goals': -1,
 'Yellow Cards': -1,
 'Bonus': 1.5,
 'Bps': 1,
 'Influence': 1,
 'Creativity': 1,
 'Threat': 1,
 'Ict Index': 1,
 'Starts': 1,
 'Expected Goal Involvements': 1.5,
 'Expected Goals Conceded': -1.5
 }

df = calculate_weighted_score(defender_scores.data.drop('Difficulty', axis=1), def_weights)
df

,Total Points,Minutes,Clean Sheets,Goals Conceded,Own Goals,Yellow Cards,Red Cards,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Expected Goal Involvements,Expected Goals Conceded,Average Difficulty,total_score
Maxence Lacroix,6.67,88.33,0.67,0.67,0.00,0.00,0.00,1.33,25.33,29.33,6.10,18.00,5.33,1.00,0.29,0.76,3.00,9.37
Daniel Muñoz,8.33,90.00,0.67,0.67,0.00,0.33,0.00,1.33,25.00,30.67,10.30,16.67,5.77,1.00,0.18,0.76,3.00,8.63
Trent Alexander-Arnold,6.33,72.67,0.67,0.67,0.00,0.00,0.00,0.00,22.33,26.93,17.93,8.00,5.27,1.00,0.13,0.83,2.33,7.83
Rayan Aït-Nouri,4.67,80.33,0.67,0.67,0.00,0.00,0.00,0.33,20.67,17.07,14.33,4.33,3.57,1.00,0.07,0.81,2.33,6.20
Andrew Robertson,3.00,84.33,0.33,1.00,0.00,0.00,0.00,0.00,12.67,13.80,25.43,7.67,4.70,1.00,0.19,0.88,2.33,5.52
Tyrick Mitchell,4.67,78.67,0.67,0.33,0.00,0.00,0.00,0.33,14.67,12.60,23.00,5.67,4.10,1.00,0.11,0.64,3.00,5.40
Marc Guéhi,5.00,90.00,0.67,0.67,0.00,0.33,0.00,0.00,23.67,28.20,5.83,8.33,4.23,1.00,0.23,0.76,3.00,5.05
Nathan Collins,5.00,90.00,0.67,0.67,0.00,0.00,0.00,0.67,20.67,26.47,1.83,3.33,3.20,1.00,0.01,0.73,3.33,4.20
Pedro Porro,4.33,68.33,0.67,0.00,0.00,0.00,0.00,0.00,16.67,21.40,18.43,2.33,4.23,0.67,0.06,1.24,3.00,3.99
Neco Williams,4.33,90.00,0.33,2.00,0.00,0.00,0.00,0.00,15.00,25.93,8.00,14.33,4.80,1.00,0.23,2.04,3.00,3.88
